<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 詞嵌入

unsup = unsupervised 非監督式

答案是自己產生的 ＝ 半監督式

深度學習必給答案, 除非你是遷移學習(偷別人的模型)

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [ ]:
import glob
import os
import textwrap
print(glob.glob(dataset))
print(os.path.dirname(dataset))
dn = os.path.dirname(dataset)
print(dn)
# print(glob.glob(dn+"/*"))
# print(glob.glob(dn+"/aclImdb/*"))
# print(glob.glob(dn+"/aclImdb/train/*"))
fn = glob.glob(dn+"/aclImdb/train/pos/*")
print(fn)
with open(fn[0], "r", encoding="utf-8") as f:
    print("pos:", textwrap.fill(f.read(), width=80))
print("="*80)
fn = glob.glob(dn+"/aclImdb/train/neg/*")
with open(fn[0], "r", encoding="utf-8") as f:
    print("neg:", textwrap.fill(f.read(), width=80))


In [ ]:
import pandas as pd
base = os.path.join(dn, "aclImdb", "train")
print(base)
def getdata(base):
    datas = {"article":[], "ans":[]}
    targets = os.path.join(base, "pos", "*")
    print(targets)
    for fn in  glob.glob(targets):
        with open(fn, "r", encoding="utf-8") as f:
            datas["article"].append(f.read())
            datas["ans"].append(1)
    targets = os.path.join(base, "neg", "*")
    for fn in glob.glob(targets):
        with open(fn, "r", encoding="utf-8") as f:
            datas["article"].append(f.read())
            datas["ans"].append(0)
    return pd.DataFrame(datas)
train = os.path.join(dn, "aclImdb", "train")
train_df = getdata(train)
test = os.path.join(dn, "aclImdb", "test")
test_df = getdata(test)

In [ ]:
print(len(train_df))
print(len(test_df))
print(test_df)
print(textwrap.fill(train_df["article"][20000]))

https://keras.io/zh/preprocessing/text/

介系詞、主詞等不需要排除掉,因為在訓練及反向傳遞時，會調整這些常用字的權重(因為在正、負面文章都會出現這些字)

In [ ]:
# Tokenize: I->1 ..love ->2.. you->3 ...
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["article"])  #fit
x_train_seq = tok.texts_to_sequences(train_df["article"])  #transform
x_test_seq = tok.texts_to_sequences(test_df["article"])  #transform
tok.index_word


In [ ]:
pd.DataFrame(x_train_seq)

In [ ]:
# 有些字消失，是因為超過最常出現的3000個單字
# tok.word_index, 0不會用到, 0 for padding
compare_vanish_words = " ".join([tok.index_word[n] for n in x_train_seq[0]])
print(textwrap.fill(compare_vanish_words, width=100))
# print(tok.sequences_to_texts(x_train_seq)[0]) #跟上面兩行效果相同, 但程式碼更簡單

tok.index_word[19]

In [ ]:
# print(tok.word_index) # 0不會用到,只能拿來padding
tok.index_word[783]

https://keras.io/zh/preprocessing/sequence/

0 就只能拿來做padding

1個單字token: 1個數字 \
1個序列sequence: 1組token(很多個數字) \



In [ ]:
MAXLEN = 1024

In [ ]:
# 截長補短 pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=MAXLEN)
x_test_pad = pad_sequences(x_test_seq, maxlen=MAXLEN)

In [ ]:
pd.DataFrame(x_train_pad)

https://keras.io/zh/layers/embeddings/

input_dim: 3000+1 (0 padding) \
output_dim:128 or 256 or 512 個情緒  \
input_length: 1024  !! \
mask_zero: 設定成true時(遮住數字0), 設0的padding填補值, 就不會被考慮進去 \



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Embedding
# 線性分類器模型(沒有ReLU、Sigmoid)
layers = [
          Embedding(3001, 128, mask_zero=True, input_length=MAXLEN),
          Flatten(),
          Dense(256, activation="relu"),
          Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
# one-hot encoding讓它自己搞定, 不需要自己做One-Hot
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])


In [ ]:
import numpy as np
y_train = np.array(train_df["ans"])
y_test = np.array(test_df["ans"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
             EarlyStopping(patience=5, restore_best_weights=True),
             ModelCheckpoint("sentiment.h5", save_best_only=True)
]
model.fit(x_train_pad,
          y_train,
          batch_size=200,
          epochs=50,
          validation_split=0.1,
          verbose=2,
          callbacks=callbacks)

In [ ]:
model.evaluate(x_test_pad, y_test)